### Import Required Libraries

In [ ]:
import numpy as np
import json
import glob
import re

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel

#spacy
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#vis
import pyLDAvis
import pyLDAvis.gensim_models
#nltk.download(punkt)
#nltk.download(wordnet) 


### Data Input 

In [ ]:

##"local_data.json" deleted from this repo for to preserve confidentiality of narratives used during training
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
stories = load_data("local_data.json")["File"]
print(stories[0][:100])


### Pre-process and vectorize the documents 

In [ ]:
#In case prompted to install stopwords, use "nltk.download(stopwords)".
#Lemmatization
stops = set(stopwords.words("english"))

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    #Use "python -m spacy download en_core_web_sm" if error [E050]
    nlp = spacy.load("en_core_web_sm",disable=["parser","ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
        
    return(texts_out)

lemmatized_texts = lemmatization(stories)
print(lemmatized_texts[0][:100])

In [ ]:
#deaccenting and removing stop words
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print (data_words[0][0:20])

In [ ]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=2,threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words],threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_trigrams(texts): 
    return(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

data_bigrams_trigrams = list(data_bigrams_trigrams)
print(data_bigrams_trigrams[:2])

In [ ]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams
print(texts[0][:20])

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

###  Initial Training and Evaluation

In [ ]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = 1  # Dont evaluate model perplexity, takes too much time

lda_model = LdaModel(
    corpus=corpus, 
    id2word=id2word,
    chunksize=chunksize,
    alpha="auto",
    eta="auto",
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

#### Save LDA Model

In [ ]:
#lda_model.save("Models/test_model_1.model")

#### Top topics detected by model (visualize for clusters)

In [ ]:
# top_topics = lda_model.top_topics(corpus)

# # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
# avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
# print(f'Average topic coherence: %.4f. % avg_topic_coherence')

# from pprint import pprint
# print(top_topics)

#### Evaluating Model

In [ ]:
new_model = gensim.models.ldamodel.LdaModel.load("Models/test_model_1.model")

In [ ]:

"""Corpus has bee taken out here, modify "Data Input" with 
new data to evaluate model.
"""

#testing the loaded model
test_doc = corpus[-1]

vector = new_model[test_doc]

#logfile = open('Evaluation/For Model V1/Story_1.txt', 'a')
# print(new_model.show_topics(num_topics=10, num_words=10, formatted=False), file = logfile)
# logfile.close()

#Sort to arrange topics based on priority or similarity. 
def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return(sub_li)
new_vector = Sort(vector)
print(new_vector)#last story is most similar with topic 2

##### Visualization

In [ ]:
#use "pip install pyLDAvis" if the pyLDAvis library isn't installed on your system
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(new_model, corpus, id2word, mds="mmds", R=30)
vis

### Tuning LDA Model

In [ ]:
#Compute Coherence Score 
coherence_model_lda = CoherenceModel(
    model = new_model,
    texts = texts, 
    dictionary = id2word, 
    coherence = 'c_v'
)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
#Using a function to find the most optimal number of topics for a better coherence score. 
"""Source_1 = https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know"""
"""Source_2 = https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#16buildingldamalletmodel"""

def compute_coherence_values(chunksize, passes, iterations, eval_every, dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
 
 -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(
        corpus=corpus, 
        id2word=id2word,
        chunksize=chunksize,
        alpha="auto",
        eta="auto",
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
        )
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(
    dictionary=id2word, 
    corpus=corpus, 
    texts=texts, 
    start=2, 
    limit=40, 
    step=4, 
    chunksize = 2000, 
    passes = 20, 
    iterations = 400,
    eval_every = 1
)

In [ ]:
# Show graph
import matplotlib.pyplot as plt 
import pandas as pd

limit=40; start=2; step=4;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
#Extracting the topic that gives the best results. 
best_result_model = coherence_values.index(max(coherence_values))
optimal_model = model_list[best_result_model]
print(f'''{x[best_result_model]} topics give the highest coherence score of {coherence_values[best_result_model]} for this specific model''')

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model, corpus, id2word, mds="mmds", R=30)
vis

In [ ]:
# optimal_model.save("Models/Model V2/test_model_2.model")

In [ ]:
results = optimal_model[corpus]
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in results]
print(corpus_topics)

In [ ]:
topics = [[(term, round(wt, 3)) for term, wt in optimal_model.show_topic(n, topn=20)] for n in range(0, optimal_model.num_topics)]
# set column width
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for term, wt in topic]) for topic in topics], columns = ['Terms per Topic'], index=['Topic'+str(t) for t in range(1, optimal_model.num_topics+1)] )
